***Ratios***
- Standalone -> https://www.moneycontrol.com/financials/itc/ratiosVI/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/consolidated-ratiosVI/ITC#ITC

***Balance Sheet***
- Standalone -> https://www.moneycontrol.com/financials/itc/balance-sheetVI/ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC
- for next page https://www.moneycontrol.com/financials/itc/profit-lossVI/ITC/2#ITC

***Profit & Loss***- 
- Standalone -> https://www.moneycontrol.com/financials/itc/profit-lossVI/ITC
- Consolidated ->https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC#ITC

***Quarterly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/quarterly-results/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-quarterly-results/ITC#ITC

***Half Yearly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/half-yearly/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-half-yearly/ITC#ITC

***Nine Months Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/nine-months/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-nine-months/ITC#ITC

***Yearly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/yearly/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-yearly/ITC#ITC

***Cash Flow***
- Standalone - https://www.moneycontrol.com/financials/itc/cash-flowVI/ITC#ITC
- Consolidated - https://www.moneycontrol.com/financials/itc/consolidated-cash-flowVI/ITC#ITC

***Capital Structure***
- https://www.moneycontrol.com/financials/itc/capital-structure/ITC#ITC

In [37]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [38]:
import pandas as pd

In [39]:
mc_fin_baseurl = 'https://www.moneycontrol.com/financials'
symbol = 'sbi'

***To download Financial ratios***

In [66]:
def processfindf(df):
    df.at[0,0] = 'item'
    df = df.iloc[:, :-1]
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    df.set_index('item', inplace=True)
    return df

# fintp -> 'ratios', 'balance-sheet', 'profit-loss', 'yearly', 'quarterly-results', 'half-yearly', 'nine-months'
def getfinancials(symbol, fintp='ratios'):
    symbol = symbol.upper()
    url = f'{mc_fin_baseurl}/{symbol}/{fintp}VI/{symbol}'
    if fintp in ['yearly', 'quarterly-results', 'half-yearly', 'nine-months']:
        url = f'{mc_fin_baseurl}/{symbol}/results/{fintp}/{symbol}'
    lst = [pd.read_html(f'{url}/{i}') for i in range(2, 6)]
    lst = [pd.read_html(url)] + lst
    dfs = [processfindf(i[0]) for i in lst if i]
    df = pd.concat(dfs, axis=1)
    df = df.dropna(how='all') 
    df.fillna(0, inplace=True)
    if fintp in ['balance-sheet', 'profit-loss']: df = df.iloc[1:]
    return df

In [72]:
%%time
# df = getfinancials('HDF01', 'ratios')
# df = getfinancials('HDF01', 'balance-sheet')
# df = getfinancials('HDF01', 'profit-loss')
# df = getfinancials('HDF01', 'yearly')
# df = getfinancials('HDF01', 'half-yearly')
# df = getfinancials('HDF01', 'quarterly-results')
# df = getfinancials('HDF01', 'nine-months')

Wall time: 31.3 s


In [76]:
%%time
symbols = ['sbi', 'itc', 'HDF01']
dfs = [getfinancials(s, 'ratios') for s in symbols]

Wall time: 1min 21s


In [79]:
df = dfs[0]

In [86]:
dfs[1]

,Mar 19,Mar 18,Mar 17,Mar 16,Mar 15,Mar 14,Mar 13,Mar 12,Mar 11,Mar 10,Mar 09,Mar 08,Mar 07,Mar 06,Mar 05,Mar 04
item,,,,,,,,,,,,,,,,
Basic EPS (Rs.),10.19,9.22,8.43,12.26,12.05,11.09,9.45,7.93,6.45,10.64,8.66,8.28,7.18,5.95,88.28,64.31
Diluted EPS (Rs.),10.13,9.16,8.38,12.20,11.93,10.96,9.33,7.84,6.45,10.64,8.66,8.28,7.18,5.95,88.28,64.31
Cash EPS (Rs.),11.24,10.13,9.25,13.52,13.19,12.18,10.40,8.78,7.29,12.23,10.11,9.44,8.14,6.84,100.89,74.07
Book Value [ExclRevalReserve]/Share (Rs.),47.27,42.12,37.33,40.85,38.28,32.95,28.14,23.97,20.55,36.69,36.24,31.85,27.59,23.97,315.63,256.35
Book Value [InclRevalReserve]/Share (Rs.),47.27,42.12,37.33,40.92,38.35,33.02,28.21,24.04,20.62,36.84,36.39,32.00,27.74,24.13,318.04,258.80
Dividend / Share(Rs.),5.75,5.15,4.75,8.50,6.25,6.00,5.25,4.50,4.45,10.00,3.70,3.50,3.10,2.65,31.00,20.00
Revenue from Operations/Share (Rs.),36.71,33.29,33.00,45.78,45.55,41.79,37.84,32.20,27.73,48.63,39.70,37.23,32.73,26.09,306.41,258.89
PBDIT/Share (Rs.),16.14,14.48,13.64,19.93,18.73,17.05,14.64,12.37,10.33,17.44,14.12,13.11,11.36,9.61,119.99,104.19
PBIT/Share (Rs.),15.07,13.54,12.78,18.65,17.53,15.92,13.63,11.48,9.48,15.85,12.66,11.94,10.39,8.73,107.38,94.44
